In [ ]:
#Muhammad Hasham Ul Haq
#i200752@nu.edu.pk
import sounddevice as sd
import numpy as np
import torchaudio
import torch
import IPython
from torchaudio.models.wav2vec2 import Wav2Vec2ForCTC
from torchaudio.transforms import Resample
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# Parameters for audio recording
sample_rate = 16000
audio_buffer = []
buffer_duration = 2  # 2-second buffer


In [ ]:
# Initialize Wav2Vec2 model for audio transcription
model, _ = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
resampler = Resample(orig_freq=sample_rate, new_freq=16000)

# Initialize Tortoise TTS for voice cloning
tts = TextToSpeech()


In [ ]:
# Function to handle incoming audio chunks
def audio_callback(indata, frames, time, status):
    if status:
        print("Error:", status)
    if any(indata):
        audio_buffer.extend(indata)
        if len(audio_buffer) > sample_rate * buffer_duration:
            process_audio()

# Function to transcribe and speak audio
def process_audio():
    audio_data = torch.FloatTensor(audio_buffer)
    audio_buffer.clear()

    # Transcribe audio
    audio_data = resampler(audio_data)
    with torch.no_grad():
        input_values = model.feature_extractor(audio_data.unsqueeze(0))
        logits = model.classifier(input_values)
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = model.tokenizer.batch_decode(predicted_ids)

    print("Transcription:", transcription)

    # Convert transcription to voice (you can choose a voice from the available ones)
    voice = 'voice_test'
    text = transcription[0]  # Use the transcription as text
    voice_samples, conditioning_latents = load_voice(voice)
    gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset="standard")
    torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
    IPython.display.Audio('generated.wav')

# Start recording live audio
with sd.InputStream(callback=audio_callback, channels=1, samplerate=sample_rate):
    print("Listening for audio...")
    sd.sleep(1000000)
